# MATRICE DI CORRELAZIONE

Come prima operazione, importiamo le librerie necessarie

In [1]:
import pandas as pd
import statistics as st
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import numpy as np
from sklearn.preprocessing import scale
from rpy2.robjects import pandas2ri

d:\dev-cpp\anaconda\lib\site-packages\rpy2\robjects\pandas2ri.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Successivamente si procede con l'importazione del dataset contenuto nella cartella "data". Per raggiungere tale scopo si utilizza il metodo **read_excel()** del modulo **pandas**

In [2]:
table = pd.read_excel('https://github.com/collab-uniba/PanelJamDataExtractor/blob/master/data/TabellaCompleta.xlsx?raw=true', index = False)

Una volta importato il dataset si devono eliminare alcune feature contenute in esso, questo perchè:

-le feature **id_prog**,**id_panel**,**panel_author** contengono delle stringhe che indicano rispettivamente, l'id dei progetti estratti, l'id dei panel contenuti in ogni progetto e il nome del creatore di ogni panel. Queste feature non sono rilevanti per il calcolo della matrice di correlazione(mostrata successivamente) e nemmeno per la costruzione del modello di regressione.

-le feature **Likes**, **Views**, **Comments** e **time** fanno riferimento ai progetti e non ai singoli panel, per questo motivo non vengono considerati.

Si procede con la costruzione della matrice di correlazione con lo scopo di osservare eventuali problemi di **multicollinearità** tra le feature. Se il grado di correlazione tra due feature è maggiore del limite consentito di **0.7**, se ne deve eliminare una delle due. La matrice verrà costruita utilizzando il metodo **corr()** del modulo **pandas**

In [3]:
#time = table['time'].tolist()
#table['time'] = list(map(lambda sub:int(''.join([ele for ele in sub if ele.isnumeric()])), time))
table = table.drop(columns = ['id_prog','id_panel','Likes','Views','Comments','panel_author','author_stars','time'])

corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,project_depth,extended,has_avatar_author,has_bio_author,author_followers,tot_loves_author,tot_views_author,author_ranking,panel_stars
project_depth,1.00,-0.42,0.06,0.03,0.02,0.04,0.05,0.07,0.07
extended,-0.42,1.00,-0.03,-0.03,-0.02,-0.03,-0.03,-0.04,-0.17
has_avatar_author,0.06,-0.03,1.00,0.24,0.18,0.15,0.11,0.09,0.08
has_bio_author,0.03,-0.03,0.24,1.00,0.22,0.12,0.01,-0.03,0.06
author_followers,0.02,-0.02,0.18,0.22,1.00,0.71,0.63,0.33,0.18
tot_loves_author,0.04,-0.03,0.15,0.12,0.71,1.00,0.76,0.31,0.19
tot_views_author,0.05,-0.03,0.11,0.01,0.63,0.76,1.00,0.63,0.12
author_ranking,0.07,-0.04,0.09,-0.03,0.33,0.31,0.63,1.00,0.12
panel_stars,0.07,-0.17,0.08,0.06,0.18,0.19,0.12,0.12,1.00


Come si può osservare dalla matrice, c'è un alto grado di correlazione tra **tot_loves_author** e **author_stars** (**0.76**) superiore al limite di **0.7**. Questo è dovuto dalla presenza di entrambe le feature nel calcolo del ranking d'autore, per questo si decide di eliminare tot_views_author. 

In [4]:
table = table.drop(columns = ['tot_views_author'])
corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,project_depth,extended,has_avatar_author,has_bio_author,author_followers,tot_loves_author,author_ranking,panel_stars
project_depth,1.00,-0.42,0.06,0.03,0.02,0.04,0.07,0.07
extended,-0.42,1.00,-0.03,-0.03,-0.02,-0.03,-0.04,-0.17
has_avatar_author,0.06,-0.03,1.00,0.24,0.18,0.15,0.09,0.08
has_bio_author,0.03,-0.03,0.24,1.00,0.22,0.12,-0.03,0.06
author_followers,0.02,-0.02,0.18,0.22,1.00,0.71,0.33,0.18
tot_loves_author,0.04,-0.03,0.15,0.12,0.71,1.00,0.31,0.19
author_ranking,0.07,-0.04,0.09,-0.03,0.33,0.31,1.00,0.12
panel_stars,0.07,-0.17,0.08,0.06,0.18,0.19,0.12,1.00


# COUNT DATA MODEL

Per costruire i count data model si uttilizza il modulo **rpy2**, il quale permette di poter importare ed utilizzare l'ambiente **R** in Python.

In [5]:
utils = importr('utils')
base = importr('base')

Installiamo tutte le librerie R necessarie. Questo può essere fatto grazie all'utilizzo del metodo **install_packages()** della libreria R **utils**.

In [ ]:
utils.install_packages('pscl')
utils.install_packages('MASS')
utils.install_packages('stats')
utils.install_packages('lmtest')
utils.install_packages('nonnest2')

Una volta installate si passa all'import delle stesse.

In [6]:
MASS = importr('MASS')
stats = importr('stats')
pscl = importr('pscl')
base = importr('base')
lmtest = importr('lmtest')
nonnest2 = importr('nonnest2')

Il primo passaggio, per la costruzione dei count data model, consiste nel dividere il dataset in due, dove il primo conterrà solo le feature non numeriche ed è stato etichettato con il nome **logit**, il secondo conterrà solo le feature numeriche ed è stato etichettato con il nome **numerical**. In seguito si spiegherà il motivo di questa divisione.

In [7]:
featureTable = table
#featureTable['time'] = numTime
logit = featureTable.drop(columns = ['project_depth','author_followers','tot_loves_author',
                'author_ranking','panel_stars'])
numerical = featureTable.drop(columns = ['extended','has_avatar_author','has_bio_author'])

bisogna effettuare una log-trasformazione delle feature numeriche in quanto alcune di esse contengono valori troppo grandi. Dopo di chè si procede alla ricostruzione della tabella, andando ad unire la tabella numerica(ottenuta dalla log-normalizzazione) alla tabella non numerica. Bisogna specificare che, dopo aver effettuato la log-trasformazione, i valori della variabile dipendente **panel_stars** vengono convertiti ad intero. Questo perchè non è possibile avere una variabile dipendente che sia di un tipo diverso da tipo intero.

In [8]:
normTable = np.log1p(numerical)
normTable['panel_stars'] = normTable['panel_stars'].astype(int)
normTable['extended'] = logit['extended']
normTable['has_avatar_author'] = logit['has_avatar_author']
normTable['has_bio_author'] = logit['has_bio_author']

A questo punto si procede con la conversione del dataframe da **pandas DataFrame** ad **R DataFrame**, questa operazione è necessaria affinchè si possa utilizzare il dataFrame. Per effettuare tale conversione si usa il metodo **pandas2ri.py2ri** del modulo rpy2. 

In [9]:
pandas2ri.activate()
rTable = pandas2ri.py2ri(normTable)

lo script successivo permette la costruzione dei count data model, in questo caso si è deciso di costruire 4 modelli, ossia **Poisson**, **Negative Binomial**, **Zero Inflated negative binomial** e **Hurdle Model**, utilizzando rispettivamente la funzione **glm** del package **stats** per il modello di Poisson, la funzione **glm_nb** del package **MASS** per la Negative Binomial, la funzione **zeroinfl** del package **pscl** per la Zero Inflated è la funzione **hurdle** del package **pscl** per Hurdle Model. La variabile **formula** contiene una stringa scritta in notazione **Patsy** utile per poter impostare la variabile dipendente (inserita a sinistra del simbolo tilde ~ ) e le variabili indipendenti che fungeranno da predittori(poste a destra del simbolo tilde). 

In [10]:
formula = """panel_stars ~ project_depth + author_followers +
             author_ranking + extended + has_avatar_author + has_bio_author + tot_loves_author"""


summary = ro.r['summary']

poissonMod = stats.glm(ro.r(formula), family = ro.r('poisson(link = "log")'), data = rTable)
nbMod = MASS.glm_nb(ro.r(formula), data = rTable)
zeroinflMod = pscl.zeroinfl(ro.r(formula), data = rTable,dist = 'negbin')
hurdleMod = pscl.hurdle(ro.r(formula), data = rTable, dist = 'negbin', zero = "binomial")

# RISULTATI COUNT DATA MODEL

**MODELLO DI POISSON**

In [11]:
poissonSum = summary(poissonMod)
loglikPoisson = stats.logLik(poissonMod)

poisson = str(poissonSum)
poisson = poisson.split('Deviance')[1]
print(poisson)
print(loglikPoisson)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-1.4258  -0.8871  -0.6865   0.6245   2.9196  



Coefficients:

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -2.38625    0.13929 -17.132  < 2e-16 ***

project_depth         -0.04333    0.03886  -1.115   0.2648    

author_followers       0.08520    0.01753   4.861 1.17e-06 ***

author_ranking         0.16503    0.01489  11.084  < 2e-16 ***

extendedTRUE          -0.45964    0.03443 -13.350  < 2e-16 ***

has_avatar_authorTRUE  0.15773    0.10118   1.559   0.1190    

has_bio_authorTRUE     0.07730    0.03243   2.383   0.0172 *  

tot_loves_author       0.08339    0.01362   6.123 9.16e-10 ***

---

Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



(Dispersion parameter for poisson family taken to be 1)



    Null deviance: 11437  on 13747  degrees of freedom

Residual deviance: 10479  on 13740  degrees of freedom

AIC: 20749



Number of Fisher Scoring iterations: 5



'

**NEGATIVE BINOMIAL**

In [12]:
nbSum = summary(nbMod)
loglikNB = stats.logLik(nbMod)

nb = str(nbSum)
nb = nb.split('Deviance')[1]
print(nb)
print(loglikNB)

 Residuals: 

    Min       1Q   Median       3Q      Max  

-1.4258  -0.8871  -0.6865   0.6245   2.9195  



Coefficients:

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -2.38627    0.13930 -17.130  < 2e-16 ***

project_depth         -0.04333    0.03886  -1.115   0.2648    

author_followers       0.08520    0.01753   4.861 1.17e-06 ***

author_ranking         0.16503    0.01489  11.084  < 2e-16 ***

extendedTRUE          -0.45964    0.03443 -13.350  < 2e-16 ***

has_avatar_authorTRUE  0.15772    0.10120   1.559   0.1191    

has_bio_authorTRUE     0.07730    0.03244   2.383   0.0172 *  

tot_loves_author       0.08339    0.01362   6.123 9.20e-10 ***

---

Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



(Dispersion parameter for Negative Binomial(8237.561) family taken to be 1)



    Null deviance: 11437  on 13747  degrees of freedom

Residual deviance: 10479  on 13740  degrees of freedom

AIC: 20751



Number of Fisher Scorin

**ZERO INFLATED NEGATIVE BINOMIAL**

In [13]:
zeroinflSum = summary(zeroinflMod)
loglikZeroinfl = stats.logLik(zeroinflMod)

zeroinfl = str(zeroinflSum)
zeroinfl = zeroinfl.split('Pearson')[1]
print(zeroinfl)
print(loglikZeroinfl)

 residuals:

    Min      1Q  Median      3Q     Max 

-0.9456 -0.6378 -0.4617  0.6907  9.3265 



Count model coefficients (negbin with log link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -1.91475    0.18944 -10.107  < 2e-16 ***

project_depth         -0.06293    0.04158  -1.513    0.130    

author_followers       0.02283    0.01981   1.152    0.249    

author_ranking         0.10546    0.01741   6.056 1.39e-09 ***

extendedTRUE          -0.41006    0.03702 -11.076  < 2e-16 ***

has_avatar_authorTRUE  0.08566    0.14771   0.580    0.562    

has_bio_authorTRUE     0.05618    0.03472   1.618    0.106    

tot_loves_author       0.11726    0.01487   7.885 3.14e-15 ***

Log(theta)            16.21801         NA      NA       NA    



Zero-inflation model coefficients (binomial with logit link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)             5.3310     1.4921   3.573 0.000353 ***

project_depth     

**HURDLE MODEL**

In [14]:
hurdleSum = summary(hurdleMod)
loglikHurdle = stats.logLik(hurdleMod)

hurdle = str(hurdleSum)
hurdle = hurdle.split('Pearson')[1]
print(hurdle)
print(loglikHurdle)

 residuals:

    Min      1Q  Median      3Q     Max 

-1.4263 -0.7262 -0.4911  0.8451  7.6005 



Count model coefficients (truncated negbin with log link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -1.40920    0.55982  -2.517   0.0118 *  

project_depth         -0.11960    0.13911  -0.860   0.3899    

author_followers       0.27035    0.06241   4.332 1.48e-05 ***

author_ranking        -0.03220    0.05219  -0.617   0.5373    

extendedTRUE          -0.57208    0.11384  -5.025 5.03e-07 ***

has_avatar_authorTRUE  0.49586    0.45762   1.084   0.2786    

has_bio_authorTRUE    -0.18502    0.10559  -1.752   0.0797 .  

tot_loves_author      -0.09076    0.04729  -1.919   0.0549 .  

Log(theta)            13.68812         NA      NA       NA    

Zero hurdle model coefficients (binomial with logit link):

                      Estimate Std. Error z value Pr(>|z|)    

(Intercept)           -2.79278    0.17494 -15.964  < 2e-16 ***

project_depth

**LIKELIHOOD RATIO TEST E TEST DI VUONG**

**Likelihood ratio test**

In [15]:
lrt = lmtest.lrtest(poissonMod,nbMod)
print(lrt)

Likelihood ratio test



Model 1: panel_stars ~ project_depth + author_followers + author_ranking + 

    extended + has_avatar_author + has_bio_author + tot_loves_author

Model 2: panel_stars ~ project_depth + author_followers + author_ranking + 

    extended + has_avatar_author + has_bio_author + tot_loves_author

  #Df LogLik Df  Chisq Pr(>Chisq)

1   8 -10366                     

2   9 -10366  1 0.1867     0.6657



**Test di Vuong**

In [16]:
nb_hurdle_vuong = nonnest2.vuongtest(hurdleMod,nbMod)
print(nb_hurdle_vuong)



Model 1 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: negbin 

 Call: (function (formula, data, weights, subset, na.action, start = NULL, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.048,   p = 4.35e-09



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 15.917,   p = <2e-16

  H1B: Model 2 fits better than Model 1 

    z = 15.917,   p = 1



In [17]:
zero_hurdle_test = nonnest2.vuongtest(zeroinflMod, hurdleMod)
print(zero_hurdle_test)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: hurdle 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.055,   p = <2e-16



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = -13.180,   p = 1

  H1B: Model 2 fits better than Model 1 

    z = -13.180,   p = < 2.2e-16



In [18]:
nb_zeroinfl_vuong = nonnest2.vuongtest(zeroinflMod,nbMod)
print(nb_zeroinfl_vuong)



Model 1 

 Class: zeroinfl 

 Call: (function (formula, data, subset, na.action, weights, offset, ...



Model 2 

 Class: negbin 

 Call: (function (formula, data, weights, subset, na.action, start = NULL, ...



Variance test 

  H0: Model 1 and Model 2 are indistinguishable 

  H1: Model 1 and Model 2 are distinguishable 

    w2 = 0.006,   p = 9.82e-05



Non-nested likelihood ratio test 

  H0: Model fits are equal for the focal population 

  H1A: Model 1 fits better than Model 2 

    z = 5.183,   p = 1.09e-07

  H1B: Model 2 fits better than Model 1 

    z = 5.183,   p = 1



# CALCOLO STANDARDIZE FACTOR CHANGE COEFFICIENTI HURDLE MODEL COUNT 

In [19]:
value = base.matrix(base.exp(stats.coef(hurdleMod, "count")))
data = {'features': ['Intercept','project_depth','author_followers','author_ranking','extended_TRUE',
                     'has_avatar_author_TRUE','has_bio_author_TRUE','tot_loves_author'],
        'SFC': value}
SFC = pd.DataFrame(data,columns = ['features','SFC'])
print(SFC)

                 features       SFC
0               Intercept  0.244339
1           project_depth  0.887272
2        author_followers  1.310428
3          author_ranking  0.968313
4           extended_TRUE  0.564348
5  has_avatar_author_TRUE  1.641914
6     has_bio_author_TRUE  0.831084
7        tot_loves_author  0.913238


# CALCOLO ODDS RATIO COEFFICIENTI HURDLE MODEL ZERO

In [20]:
values = base.matrix((base.exp(stats.coef(hurdleMod, "zero"))))
oddsRatio = []
for coef in values:
    oddsRatio.append(1/coef)

data = {'features': ['Intercept','project_depth','author_followers','author_ranking','extended_TRUE',
                     'has_avatar_author_TRUE','has_bio_author_TRUE','tot_loves_author'],
        'Odds Ratio': oddsRatio}
oddsRatioTble = pd.DataFrame(data,columns = ['features','Odds Ratio'])
print(oddsRatioTble)

                 features  Odds Ratio
0               Intercept   16.326389
1           project_depth    1.058255
2        author_followers    0.885105
3          author_ranking    0.750231
4           extended_TRUE    2.103365
5  has_avatar_author_TRUE    0.922879
6     has_bio_author_TRUE    0.858898
7        tot_loves_author    0.878080
